<a href="https://colab.research.google.com/github/nMaax/danteGPT/blob/main/dante_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/nMaax/danteGPT/main/divina_commedia.txt

--2025-03-15 21:41:03--  https://raw.githubusercontent.com/nMaax/danteGPT/main/divina_commedia.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 448183 (438K) [text/plain]
Saving to: ‘divina_commedia.txt.1’

divina_commedia.txt 100%[===================>] 437.68K  --.-KB/s    in 0.004s  

2025-03-15 21:41:03 (102 MB/s) - ‘divina_commedia.txt.1’ saved [448183/448183]



In [2]:
# Read the file
with open('divina_commedia.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(text[:10000])

INFERNO CANTO 1
Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura
ché la diritta via era smarrita.
Ahi quanto a dir qual era è cosa dura
esta selva selvaggia e aspra e forte
che nel pensier rinova la paura!
Tant’ è amara che poco è più morte;
ma per trattar del ben ch’i’ vi trovai,
dirò de l’altre cose ch’i’ v’ho scorte.
Io non so ben ridir com’ i’ v’intrai,
tant’ era pien di sonno a quel punto
che la verace via abbandonai.
Ma poi ch’i’ fui al piè d’un colle giunto,
là dove terminava quella valle
che m’avea di paura il cor compunto,
guardai in alto, e vidi le sue spalle
vestite già de’ raggi del pianeta
che mena dritto altrui per ogne calle.
Allor fu la paura un poco queta
che nel lago del cor m’era durata
la notte ch’i’ passai con tanta pieta.
E come quei che con lena affannata
uscito fuor del pelago a la riva
si volge a l’acqua perigliosa e guata,
così l’animo mio, ch’ancor fuggiva,
si volse a retro a rimirar lo passo
che non lasciò già mai persona viva.
Poi ch’èi 

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
print(''.join(chars))
print(vocab_size)


 !'(),-.0123456789:;?ABCDEFGHILMNOPQRSTUVZabcdefghijlmnopqrstuvxyz «»ÈËÏàäèéëìïòóöùü–—‘’“”…﻿
93


In [6]:
stoi = {ch:i for i,ch in enumerate(chars)}

In [7]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

In [8]:
encode = lambda x : [stoi[ch] for ch in x]
decode = lambda x : ''.join([itos[i] for i in x])

In [9]:
print(encode('Nel mezzo del cammin'))

[33, 47, 53, 1, 54, 47, 66, 66, 56, 1, 46, 47, 53, 1, 45, 43, 54, 54, 51, 55]


In [10]:
import torch

In [11]:
data = torch.tensor(encode(text), dtype=torch.long)

In [12]:
print(data.shape, data.dtype)

torch.Size([416980]) torch.int64


In [13]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [14]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  print("context:", x[:t+1])
  print("target: ", y[t].item())

context: tensor([30])
target:  33
context: tensor([30, 33])
target:  27
context: tensor([30, 33, 27])
target:  26
context: tensor([30, 33, 27, 26])
target:  37
context: tensor([30, 33, 27, 26, 37])
target:  33
context: tensor([30, 33, 27, 26, 37, 33])
target:  34
context: tensor([30, 33, 27, 26, 37, 33, 34])
target:  1
context: tensor([30, 33, 27, 26, 37, 33, 34,  1])
target:  24


In [15]:
torch.manual_seed(42)

def get_batch(split='train', batch_size=32, block_size=8):
  data = train_data if split=='train' else test_data
  ix = torch.randint(low=0, high=(len(data) - block_size), size=(batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  y = torch.clamp(y, 0, vocab_size - 1)
  return x, y

In [16]:
xb, yb = get_batch('train', batch_size=4, block_size=8)

In [17]:
print(xb.shape, xb, sep='\n')
print(yb.shape, yb, sep='\n')

torch.Size([4, 8])
tensor([[48, 59, 47, 46, 46, 56,  1, 43],
        [47,  6,  1, 55, 56, 55,  1, 60],
        [49, 56, 54, 47, 55, 61, 51, 19],
        [51, 43,  1, 61, 47, 49, 49, 50]])
torch.Size([4, 8])
tensor([[59, 47, 46, 46, 56,  1, 43, 55],
        [ 6,  1, 55, 56, 55,  1, 60, 43],
        [56, 54, 47, 55, 61, 51, 19,  1],
        [43,  1, 61, 47, 49, 49, 50, 51]])


In [18]:
xb, yb = get_batch('train')
for b in range(8):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when the context is {context} then the target is {target}")

when the context is tensor([63]) then the target is 51
when the context is tensor([63, 51]) then the target is 47
when the context is tensor([63, 51, 47]) then the target is 61
when the context is tensor([63, 51, 47, 61]) then the target is 56
when the context is tensor([63, 51, 47, 61, 56]) then the target is 6
when the context is tensor([63, 51, 47, 61, 56,  6]) then the target is 0
when the context is tensor([63, 51, 47, 61, 56,  6,  0]) then the target is 46
when the context is tensor([63, 51, 47, 61, 56,  6,  0, 46]) then the target is 51
when the context is tensor([54]) then the target is 51
when the context is tensor([54, 51]) then the target is 49
when the context is tensor([54, 51, 49]) then the target is 53
when the context is tensor([54, 51, 49, 53]) then the target is 51
when the context is tensor([54, 51, 49, 53, 51]) then the target is 43
when the context is tensor([54, 51, 49, 53, 51, 43]) then the target is 55
when the context is tensor([54, 51, 49, 53, 51, 43, 55]) the

In [19]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class naiveLM(nn.Module):
  def __init__(self, vocab_size, latent_space_dim):
    super().__init__()
    self.embedding_table = nn.Embedding(vocab_size, latent_space_dim)
    self.net = nn.Sequential(
        nn.Linear(in_features=latent_space_dim, out_features=latent_space_dim, bias=True),
        nn.ReLU(),
        nn.Linear(in_features=latent_space_dim, out_features=latent_space_dim, bias=True),
        nn.ReLU(),
        nn.Linear(in_features=latent_space_dim, out_features=latent_space_dim, bias=True),
        nn.ReLU(),
        nn.Linear(in_features=latent_space_dim, out_features=vocab_size, bias=True)
    )

  def forward(self, idx):
    emb = self.embedding_table(idx)
    logits = self.net(emb)
    return logits

  def compute_loss(self, idx, targets):
    logits = self.forward(idx)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)
    return loss

  def generate(self, idx, max_new_tokens=32):
    for _ in range(max_new_tokens):
      logits = self.forward(idx) # B, T, C
      logits = logits[:, -1, :] # B, C
      probs = F.softmax(logits, dim=-1) # B, C
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # idx: (B, T) --> (B, T+1)
    return idx

In [20]:
model = naiveLM(vocab_size=vocab_size, latent_space_dim=32)

In [21]:
print(xb.shape, xb, sep='\n')

torch.Size([32, 8])
tensor([[63, 51, 47, 61, 56,  6,  0, 46],
        [54, 51, 49, 53, 51, 43, 55, 61],
        [43, 46, 51,  1, 51, 55, 55, 43],
        [ 1, 53, 43,  1, 54, 47, 55, 61],
        [ 1, 24, 51, 59, 56, 19,  0, 68],
        [45, 45, 50, 51, 56,  8,  0, 24],
        [50, 51, 62, 46, 47, 20,  1, 47],
        [ 1, 57, 62, 59,  1, 45, 50, 88],
        [54, 47,  0, 60, 56, 61, 61, 56],
        [56,  1, 61, 59, 43,  1, 45, 56],
        [56, 54, 43, 55, 46, 56, 53, 53],
        [ 0, 63, 51, 46, 51,  1, 60, 78],
        [56, 45, 56,  1, 46, 51,  1, 63],
        [51,  1, 58, 62, 47, 53, 53, 56],
        [ 1, 61, 47,  1, 46, 43,  1, 61],
        [ 1, 57, 51, 83,  1, 53, 56,  1],
        [ 1, 51, 55, 61, 43, 49, 53, 51],
        [47, 61, 43,  6,  1, 68, 51, 56],
        [43,  1, 26, 58, 62, 43, 61, 56],
        [62, 56, 51,  1, 47, 61, 61, 47],
        [43,  1, 55, 88, 47, 59, 43,  1],
        [61, 61, 47,  1, 46, 88, 43, 54],
        [ 8,  0, 24, 51, 80,  1, 45, 50],
        [53, 4

In [22]:
logits = model.forward(xb)

In [23]:
print(logits.shape, logits, sep='\n')

torch.Size([32, 8, 93])
tensor([[[ 1.7708e-02, -9.5454e-02, -6.3445e-02,  ...,  2.8933e-02,
           1.2723e-01,  7.2484e-02],
         [ 4.0687e-02, -1.5867e-01, -9.0586e-02,  ...,  2.8630e-02,
           1.6466e-01,  1.2058e-02],
         [-6.1758e-03, -1.2827e-01, -6.2576e-02,  ...,  6.6480e-03,
           9.5547e-02,  9.9728e-02],
         ...,
         [-2.3439e-03, -8.5113e-02, -7.8333e-02,  ..., -1.1967e-02,
           1.6515e-01,  1.2914e-04],
         [-4.0942e-02, -7.1079e-02, -7.6534e-02,  ...,  1.6888e-02,
           1.5284e-01,  5.9717e-03],
         [ 3.3854e-02, -1.0048e-01, -8.2002e-02,  ...,  5.4976e-03,
           1.5675e-01,  2.9277e-02]],

        [[ 1.2351e-02, -8.8411e-02, -9.7834e-02,  ...,  1.4341e-02,
           2.0603e-01, -2.7528e-02],
         [ 4.0687e-02, -1.5867e-01, -9.0586e-02,  ...,  2.8630e-02,
           1.6466e-01,  1.2058e-02],
         [-6.5279e-03, -9.1783e-02, -6.9481e-02,  ...,  3.1910e-03,
           1.4682e-01,  2.3473e-02],
         ...,
 

In [24]:
print(xb.shape, xb, sep='\n')

torch.Size([32, 8])
tensor([[63, 51, 47, 61, 56,  6,  0, 46],
        [54, 51, 49, 53, 51, 43, 55, 61],
        [43, 46, 51,  1, 51, 55, 55, 43],
        [ 1, 53, 43,  1, 54, 47, 55, 61],
        [ 1, 24, 51, 59, 56, 19,  0, 68],
        [45, 45, 50, 51, 56,  8,  0, 24],
        [50, 51, 62, 46, 47, 20,  1, 47],
        [ 1, 57, 62, 59,  1, 45, 50, 88],
        [54, 47,  0, 60, 56, 61, 61, 56],
        [56,  1, 61, 59, 43,  1, 45, 56],
        [56, 54, 43, 55, 46, 56, 53, 53],
        [ 0, 63, 51, 46, 51,  1, 60, 78],
        [56, 45, 56,  1, 46, 51,  1, 63],
        [51,  1, 58, 62, 47, 53, 53, 56],
        [ 1, 61, 47,  1, 46, 43,  1, 61],
        [ 1, 57, 51, 83,  1, 53, 56,  1],
        [ 1, 51, 55, 61, 43, 49, 53, 51],
        [47, 61, 43,  6,  1, 68, 51, 56],
        [43,  1, 26, 58, 62, 43, 61, 56],
        [62, 56, 51,  1, 47, 61, 61, 47],
        [43,  1, 55, 88, 47, 59, 43,  1],
        [61, 61, 47,  1, 46, 88, 43, 54],
        [ 8,  0, 24, 51, 80,  1, 45, 50],
        [53, 4

In [25]:
print(yb.shape, yb, sep='\n')

torch.Size([32, 8])
tensor([[51, 47, 61, 56,  6,  0, 46, 51],
        [51, 49, 53, 51, 43, 55, 61, 47],
        [46, 51,  1, 51, 55, 55, 43, 55],
        [53, 43,  1, 54, 47, 55, 61, 47],
        [24, 51, 59, 56, 19,  0, 68, 38],
        [45, 50, 51, 56,  8,  0, 24, 56],
        [51, 62, 46, 47, 20,  1, 47,  1],
        [57, 62, 59,  1, 45, 50, 88, 51],
        [47,  0, 60, 56, 61, 61, 56,  1],
        [ 1, 61, 59, 43,  1, 45, 56, 61],
        [54, 43, 55, 46, 56, 53, 53, 56],
        [63, 51, 46, 51,  1, 60, 78,  1],
        [45, 56,  1, 46, 51,  1, 63, 51],
        [ 1, 58, 62, 47, 53, 53, 56,  1],
        [61, 47,  1, 46, 43,  1, 61, 62],
        [57, 51, 83,  1, 53, 56,  1, 88],
        [51, 55, 61, 43, 49, 53, 51, 43],
        [61, 43,  6,  1, 68, 51, 56,  1],
        [ 1, 26, 58, 62, 43, 61, 56, 59],
        [56, 51,  1, 47, 61, 61, 47, 59],
        [ 1, 55, 88, 47, 59, 43,  1, 60],
        [61, 47,  1, 46, 88, 43, 54, 56],
        [ 0, 24, 51, 80,  1, 45, 50, 47],
        [43,  

In [26]:
loss = model.compute_loss(xb, yb)

In [27]:
print(loss)

tensor(4.5208, grad_fn=<NllLossBackward0>)


In [28]:
print(xb)

tensor([[63, 51, 47, 61, 56,  6,  0, 46],
        [54, 51, 49, 53, 51, 43, 55, 61],
        [43, 46, 51,  1, 51, 55, 55, 43],
        [ 1, 53, 43,  1, 54, 47, 55, 61],
        [ 1, 24, 51, 59, 56, 19,  0, 68],
        [45, 45, 50, 51, 56,  8,  0, 24],
        [50, 51, 62, 46, 47, 20,  1, 47],
        [ 1, 57, 62, 59,  1, 45, 50, 88],
        [54, 47,  0, 60, 56, 61, 61, 56],
        [56,  1, 61, 59, 43,  1, 45, 56],
        [56, 54, 43, 55, 46, 56, 53, 53],
        [ 0, 63, 51, 46, 51,  1, 60, 78],
        [56, 45, 56,  1, 46, 51,  1, 63],
        [51,  1, 58, 62, 47, 53, 53, 56],
        [ 1, 61, 47,  1, 46, 43,  1, 61],
        [ 1, 57, 51, 83,  1, 53, 56,  1],
        [ 1, 51, 55, 61, 43, 49, 53, 51],
        [47, 61, 43,  6,  1, 68, 51, 56],
        [43,  1, 26, 58, 62, 43, 61, 56],
        [62, 56, 51,  1, 47, 61, 61, 47],
        [43,  1, 55, 88, 47, 59, 43,  1],
        [61, 61, 47,  1, 46, 88, 43, 54],
        [ 8,  0, 24, 51, 80,  1, 45, 50],
        [53, 43,  1, 63, 56, 45, 4

In [29]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


,ìp…üüCVj ;
…Ë–3e﻿5Rù»»E7èFìBùGmILSÈ2xÏ.n“3rrb:?èii–4vaN﻿NÈ(ìï«…R﻿Q””hF)F»n9–﻿ög«ayf—ù8ón4—?ZÏ9“äOiàlG6﻿,pEéPqm e;èpUï-èADnA-è0(–»om6…Ré.g8 vH8…p;1B2ëmZ 7;jZBöt
FùÏä»ù4ìQìb’px—FMPbvzu»Vx5GZvÈ:P:hi:ëómäF﻿ ;ëb﻿G–3-TàAFa'CO»u6FqëxB-4ò”Ëvu… 8dqìfïaö24rï;ëqiPès2TQtA–3Tövs2V–4Ïf—,8a5CbREÈja-Iì8Ëb–-8fAyM9FÈ’ì8uËë)7ìmZqïüZVÏ(ifü–2S‘e–?i-5xL?g?–she?ì9azöU)»äcVSzèq;?geSë«,Zó3Ppàx;q lùx1 :9r8ö6x-1yGPTb7n44eóóébzzZ0T1Nq9Zb'ù–﻿ é8ËU﻿‘“ òàj ùNïaoO0﻿'ìpz?hLn'lyR:i7eRy“ò‘FQzM8ùncòt;öÈ‘l–ürIëNùxbìODïö…lyé;ü“p﻿﻿8


In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
epochs = 10000
for i in range(epochs):

  xb, yb = get_batch('loss', batch_size=32, block_size=8)

  loss = model.compute_loss(xb, yb)
  if i % 1000 == 0: print(loss.item())
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()


4.52573299407959
2.4445390701293945
2.27592396736145
2.3547680377960205
2.144325017929077
2.4452457427978516
2.264052152633667
2.2559070587158203
2.2361109256744385
2.373514413833618


In [31]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


tuante',
ci delerin pri sonchesevie coranere co ardisue lo donto co sego.
crdeba ch’ir co ch’ gumenzelordera vutomer de;
E e.
nderda lal Bëa fll to pe,
po tinti ndor ’ l stomeravara mo: mepe, alal cimpo.
ineddi luni o ligami
E emontal l’ co mma l chesatarco,
E cora, aci fatudo l r sì a «el dar prton diche amettte.
lurv dedi chere fle na,
er ide l’uave voci qua
die ’ertte:
pomarei
ce liso fua nza, peron mpen simbire, splà cheme
siù pari ie va ne E tonoll’ive;
pre tovido one
diss co ttespe i’aconz


In [32]:
# SELF ATTENTION DEFINITION

torch.manual_seed(42)

B, T, C = 4, 8, 32
x = torch.randn(B, T, C) # Embedded batches of senteces

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

K = key(x)
Q = query(x)
V = value(x)

# "sa" = "self attention"
log_sa = Q @ K.transpose(-1, -2)
slog_sa = log_sa * head_size**-0.5 # "s" = "scaled"

tril = torch.tril(torch.ones(T,T))
autoregressive_slog_sa = slog_sa.masked_fill(tril == 0, float('-inf'))
self_attention = F.softmax(autoregressive_slog_sa, dim=-1)

out = self_attention @ V

In [33]:
self_attention[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4106, 0.5894, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3657, 0.2283, 0.4061, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2168, 0.2759, 0.2204, 0.2870, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2553, 0.1697, 0.1548, 0.2341, 0.1861, 0.0000, 0.0000, 0.0000],
        [0.1318, 0.2060, 0.1405, 0.1917, 0.1949, 0.1351, 0.0000, 0.0000],
        [0.2137, 0.0978, 0.2374, 0.1025, 0.1418, 0.0838, 0.1230, 0.0000],
        [0.0852, 0.1047, 0.0824, 0.1376, 0.1015, 0.1900, 0.1780, 0.1206]],
       grad_fn=<SelectBackward0>)

In [34]:
out.shape

torch.Size([4, 8, 16])

In [35]:
class Head(nn.Module):
  def __init__(self, feat_in, head_size):
    super().__init__()
    self.feat_in = feat_in
    self.head_size = head_size
    self.key = nn.Linear(feat_in, head_size, bias=False)
    self.query = nn.Linear(feat_in, head_size, bias=False)
    self.value = nn.Linear(feat_in, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, X):

    B, T, C = X.shape

    K = self.key(X)
    Q = self.query(X)
    V = self.value(X)

    sa = K @ Q.transpose(-1, -2) * self.head_size**-0.5
    sa = sa.masked_fill(torch.tril(torch.ones(T, T, device=X.device) == 0), float('-inf'))
    sa = F.softmax(sa, dim=-1)

    return sa @ V

In [36]:
class MultiHead(nn.Module):
  def __init__(self, num_heads, latent_space_dim):
    super().__init__()
    head_size = latent_space_dim // num_heads
    self.num_heads = num_heads
    self.heads = nn.ModuleList([
        Head(feat_in=latent_space_dim, head_size=head_size) for _ in range(num_heads)
        ])
    #self.proj = nn.Linear(latent_space_dim, latent_space_dim)

  def forward(self, X):
    out = torch.cat([h(X) for h in self.heads], dim=-1)
    #out = self.proj(out)
    return out

In [37]:
class FeedForward(nn.Module):
  def __init__(self, latent_space_dim, hidden_layer_expansion_factor=4):
    super().__init__()
    self.latent_space_dim = latent_space_dim
    self.hidden_layer_expansion_factor = hidden_layer_expansion_factor
    self.net = nn.Sequential(
            nn.Linear(latent_space_dim, hidden_layer_expansion_factor*latent_space_dim),
            nn.ReLU(),
            nn.Linear(hidden_layer_expansion_factor*latent_space_dim, latent_space_dim),
        )

  def forward(self, X):
    return self.net(X)

In [38]:
class Block(nn.Module):
  def __init__(self, latent_space_dim, num_heads):
    super().__init__()
    self.self_attention = MultiHead(num_heads=num_heads, latent_space_dim=latent_space_dim)
    self.ff = FeedForward(latent_space_dim=latent_space_dim)
    self.ln1 = nn.LayerNorm(latent_space_dim)
    self.ln2 = nn.LayerNorm(latent_space_dim)

  def forward(self, X):
    X = X + self.self_attention(self.ln1(X))
    X = X + self.ff(self.ln2(X))
    return X

In [39]:
class DanteLM(nn.Module):
  def __init__(self, vocab_size, context_window_size, latent_space_dim, num_heads, n_blocks):
    super().__init__()

    self.vocab_size = vocab_size
    self.context_window_size = context_window_size
    self.latent_space_dim = latent_space_dim
    self.num_heads = num_heads
    self.n_blocks = n_blocks

    self.tok_embedding_table = nn.Embedding(vocab_size, latent_space_dim)
    self.pos_embedding_table = nn.Embedding(context_window_size, latent_space_dim)
    self.blocks = nn.Sequential(
        *[Block(latent_space_dim, num_heads) for _ in range(n_blocks)]
    )
    self.ln_f = nn.LayerNorm(latent_space_dim)
    self.lm_head = nn.Linear(latent_space_dim, vocab_size)

  def forward(self, idx):
    B, T = idx.shape

    tok_emb = self.tok_embedding_table(idx) # (B, T, C)
    pos_emb = self.pos_embedding_table(torch.arange(T, device=idx.device)) # (T,C)
    emb = tok_emb + pos_emb # (B, T, C)
    x = self.blocks(emb)
    x = self.ln_f(x) # (B,T,C)
    logits = self.lm_head(x) # (B,T,vocab_size)
    return logits

  def compute_loss(self, idx, targets):
    logits = self.forward(idx)
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    targets = targets.view(B*T)
    loss = F.cross_entropy(logits, targets)
    return loss

  def generate(self, idx, max_new_tokens=32):
    for _ in range(max_new_tokens):
      logits = self.forward(idx[:, -self.context_window_size:]) # B, T, C
      logits = logits[:, -1, :] # B, C
      probs = F.softmax(logits, dim=-1) # B, C
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      idx = torch.cat((idx, idx_next), dim=1) # idx: (B, T) --> (B, T+1)
    return idx

In [40]:
Dante = DanteLM(vocab_size=vocab_size, context_window_size=8, latent_space_dim=32, num_heads=4, n_blocks=4)

In [41]:
# Check for GPU availability and move model and data
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Training on:", device)
else:
    device = torch.device("cpu")
    print("GPU not available. Training on CPU.")

model = Dante.to(device)  # Move your model to the device

GPU is available. Training on: cuda


In [42]:
print(decode(Dante.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=32)[0].tolist()))


ä
0r ÈQ“IAöÈE7ù)7jM-z-”5 TéNzZ0I


In [43]:
optimizer = torch.optim.AdamW(Dante.parameters(), lr=1e-3)
epochs = 100000
for i in range(epochs):

  xb, yb = get_batch('loss', batch_size=32, block_size=Dante.context_window_size)
  # Move xb and yb to the device and reassign them
  xb = xb.to(device)
  yb = yb.to(device)

  loss = Dante.compute_loss(xb, yb)
  if i % 10000 == 0: print(loss.item())
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

4.700807094573975
0.24238070845603943
0.2518538236618042
0.2248467206954956
0.13284353911876678
0.2427430897951126
0.15608833730220795
0.148853600025177
0.12976951897144318
0.16685450077056885


In [44]:
print(decode(Dante.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=500)[0].tolist()))



:«
1
Sparore corrà già l’etate,
non sterno fatta si convenne.
A quanto paradise,
in venni
quei e avvvivile.
La s’acccende e accolor incoro a l’unffe,
e sì spirava,
e al serà uo ragionne si mastui senternagnon si sin tieto
l’eplezza:
che lüa dre è natura fron ’incosì il giarri si dissinzyezzòa
ma intelletto da l’altrucide
sume si cielo e cota coma primo al contesi
canti intrato sì com’ io conco’nevie: «Ma, com’ io mosse per lo sdov’

più ch’i’ vedrai ch’esser l’altro a gncrea, perse non tre rimo
